In [ ]:
import networkx as nx
%matplotlib inline

# Chapter 2 Tutorial

多くの練習問題の後に `assert` 文を含むブロックがあることに注意されたい。この `assert` 文の前にセットアップコードがある場合がある。これはあなたが正しい道にいるというフィードバックを与えるために設置されている。もし `AssertionError` を受け取ったら、おそらくどこかに間違いがあるということだ。

Contents:

1. 経路（Paths）
2. 連結成分（Connected components）
3. 有向路（Directed paths & components）
4. Dataset: US air traffic network